In [ ]:
#1. 데이터 읽어오기
import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

In [2]:
#2. 데이터 정제 - preprocess_sentence() 사용
corpus= []
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜀
    if ('[' and ']') in sentence: continue # 대괄호가 들어가는 문장 제외

    corpus.append(sentence)
        
corpus = list(set(corpus)) 
print(f'중복되는 문장 제외한 데이터 크기 : {len(corpus)}')
print(f'원래 데이터 크기 : {len(raw_corpus)}')
print(f'공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : {len(corpus)}')

중복되는 문장 제외한 데이터 크기 : 117114
원래 데이터 크기 : 187088
공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : 117114


In [3]:
#전처리 함수 정의
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
corpus_tokenized = []

for sentence in corpus:
    # 데이터 정제
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus_tokenized.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인
corpus_tokenized[:10]
        

corpus[:10]

['But hey now, you know girl,',
 'When someone in the dark reaches out for you',
 'With the Ray Beons',
 'What makes the wind wanna blow tonight?',
 'Kun mina vien sen markniniolle',
 "There doesn't ever have to be disaster",
 "I'm going away, I'm going away",
 "(Let's get married) Let me say",
 'Love on another (love one another)',
 'From all the jails the boys and girls']

In [5]:
def tokenize(corpus):
   
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
  
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_tokenized)

[[   2   31  137 ...    0    0    0]
 [   2   45  310 ...    0    0    0]
 [   2   30    6 ...    0    0    0]
 ...
 [1339 2909  125 ...  512 3890    3]
 [   2   51    4 ...    0    0    0]
 [   2   83   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa5e86d6670>


In [6]:
# tokenizer에 구축된 단어 사전의 인덱스
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  31 137  50   4   7  36  85   4   3   0   0   0   0]
[ 31 137  50   4   7  36  85   4   3   0   0   0   0   0]


In [8]:
#평가 데이터셋 분리
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,       # 데이트셋 비율
                                                          shuffle=True, 
                                                          random_state=34)     # 결과를 일정하게 보여주기위해 지정

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (93691, 14)
Target Train: (93691, 14)


In [10]:
from tensorflow.keras.layers import Embedding, LSTM, Dense


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
2928/2928 [==============================] - 251s 79ms/step - loss: 3.3122
Epoch 2/10
2928/2928 [==============================] - 231s 79ms/step - loss: 2.9183
Epoch 3/10
2928/2928 [==============================] - 231s 79ms/step - loss: 2.6843
Epoch 4/10
2928/2928 [==============================] - 231s 79ms/step - loss: 2.4214
Epoch 5/10
2928/2928 [==============================] - 232s 79ms/step - loss: 2.1179
Epoch 6/10
2928/2928 [==============================] - 231s 79ms/step - loss: 1.8056
Epoch 7/10
2928/2928 [==============================] - 231s 79ms/step - loss: 1.5285
Epoch 8/10
2928/2928 [==============================] - 232s 79ms/step - loss: 1.3161
Epoch 9/10
2928/2928 [==============================] - 232s 79ms/step - loss: 1.1814
Epoch 10/10
2928/2928 [==============================] - 231s 79ms/step - loss: 1.1043


In [12]:
#모델 평가

# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환함.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 함. 
    while True:
        predict = model(test_tensor)  
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   
    
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
  
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated  

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so <end> '

<회고>
- 중복되는 문장, 대괄호가 포함된 문장 삭제 등 데이터 정제 시행함.
- 15000개의 단어를 기억할 수 있는 tokenzier를 생성함.
- hyperparameter에 대한 이해가 부족하여 충분한 이해가 필요함.